In [105]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin

In [109]:
class StdScalerByGroup(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # X may not be a pandas dataframe (e.g. a np.array)
        df = pd.DataFrame(X)
        
        # A dictionary of means/standard-deviations for each column, for each group.
        dict_out = {}
        for i in set(df[df.columns[0]]):
            dict_mid = {}
            temp = df[df[df. columns[0]] == i]
            for j in (temp.columns[1:]):
                dict_in = {"mean": temp[j].mean(), "std": temp.std()[j]}
                dict_mid[j] = dict_in
            dict_out[i] = dict_mid
        self.grps_ = dict_out

        return self

    def transform(self, X, y=None):
        try:
            getattr(self, "grps_")
        except AttributeError:
            raise RuntimeError("You must fit the transformer before tranforming the data!")
        
        # Define a helper function here?
        def hell0(a,b, col):
            return (b - self.grps_[a][col]["mean"]) / self.grps_[a][col]["std"]
        # X may not be a dataframe (e.g. np.array)

        df = pd.DataFrame(X)
        res = pd.DataFrame()
        for i in df.columns[1:]:
            res[i] = list(map(hell0, np.array(df[df.columns[0]]), np.array(df[i]), [i] * len(df[i])))
        return res

In [110]:
titanic = pd.read_csv('train.csv')
titanic['Name'] = titanic.Name.apply(lambda x:x.split(',')[1].split('.')[0] if x is not np.NaN else 'NAN')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S


In [113]:
titanic['Age'] = titanic.Age.fillna(df.Age.mean())
titanic['Cabin'] = titanic.Cabin.fillna('NAN')
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Mr,male,22.00000,1,0,A/5 21171,7.2500,NAN,S
1,2,1,1,Mrs,female,38.00000,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Miss,female,26.00000,0,0,STON/O2. 3101282,7.9250,NAN,S
3,4,1,1,Mrs,female,35.00000,1,0,113803,53.1000,C123,S
4,5,0,3,Mr,male,35.00000,0,0,373450,8.0500,NAN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,Rev,male,27.00000,0,0,211536,13.0000,NAN,S
887,888,1,1,Miss,female,19.00000,0,0,112053,30.0000,B42,S
888,889,0,3,Miss,female,30.27259,1,2,W./C. 6607,23.4500,NAN,S
889,890,1,1,Mr,male,26.00000,0,0,111369,30.0000,C148,C


In [119]:
y_train = titanic.Survived
x_train = titanic.drop(['PassengerId', 'Ticket', 'Survived'], axis = 1)
x_train

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,Mr,male,22.00000,1,0,7.2500,NAN,S
1,1,Mrs,female,38.00000,1,0,71.2833,C85,C
2,3,Miss,female,26.00000,0,0,7.9250,NAN,S
3,1,Mrs,female,35.00000,1,0,53.1000,C123,S
4,3,Mr,male,35.00000,0,0,8.0500,NAN,S
...,...,...,...,...,...,...,...,...,...
886,2,Rev,male,27.00000,0,0,13.0000,NAN,S
887,1,Miss,female,19.00000,0,0,30.0000,B42,S
888,3,Miss,female,30.27259,1,2,23.4500,NAN,S
889,1,Mr,male,26.00000,0,0,30.0000,C148,C
